In [ ]:
'''
Generate the dect projection data set. 3 layer mean
'''

In [2]:
import pandas as pd
import SimpleITK as sitk
import h5py
import os
import numpy as np
import glob

In [10]:
input_dir = '/home/dwu/data/DECT/sinogram/'

postfix = 'half'
output_dir = '/home/dwu/trainData/deep_denoiser_ensemble/data/dect_2d_3_layer_mean/prj_%s'%postfix
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

nlayers = 3
nslices = 100
    
if postfix == 'full':
    filenames = glob.glob(os.path.join(input_dir, 'sino_*_1.mat'))
else:
    filenames = glob.glob(os.path.join(input_dir, 'sino_*_2.mat'))

In [11]:
def pad_projection_b(prj_a, prj_b, n_pixels = 20, reg = 0.1):
    margin = int((prj_a.shape[2] - prj_b.shape[2]) / 2)
    
    prj_b_left = prj_b[..., :n_pixels]
    prj_a_left = prj_a[..., margin:margin+n_pixels]
    coefs_left = np.sum(prj_b_left * prj_a_left, 2) / np.sum(prj_b_left * prj_a_left + reg, 2)
    
    prj_b_right = prj_b[..., -n_pixels:]
    prj_a_right = prj_a[..., -margin-n_pixels:-margin]
    coefs_right = np.sum(prj_b_right * prj_a_right, 2) / np.sum(prj_b_right * prj_a_right + reg, 2)
    
    extrap_prj_b = np.zeros_like(prj_a)
    extrap_prj_b[..., :margin] = prj_a[..., :margin] * coefs_left[..., np.newaxis]
    extrap_prj_b[..., margin:-margin] = prj_b
    extrap_prj_b[..., -margin:] = prj_a[..., -margin:] * coefs_right[..., np.newaxis]
    
    return extrap_prj_b

In [25]:
np.random.seed(0)
manifest = []
print (len(filenames))
for k, filename in enumerate(filenames[13:]):
    print (k, filename)
    
    name = os.path.basename(filename)[:-4]
    
    with h5py.File(filename, 'r') as f:
        prj_a = np.copy(f['sinoA']).astype(np.float32)
        prj_b = np.copy(f['sinoB']).astype(np.float32)
    
    ext_b = pad_projection_b(prj_a, prj_b)
    
    # select the layers
    inds = np.arange(0, len(prj_a) + 1 - nlayers)
    inds = np.random.choice(inds, nslices, False)
    inds = np.sort(inds)
    
    # calculate 3-layer mean
    selected_a = []
    selected_b = []
    for i in inds:
        manifest.append({'Tag': name, 'Slice': i})
        selected_a.append(prj_a[i:i+nlayers].mean(0))
        selected_b.append(ext_b[i:i+nlayers].mean(0))
    selected_a = np.array(selected_a).astype(np.float32)
    selected_b = np.array(selected_b).astype(np.float32)
    
    sitk_a = sitk.GetImageFromArray(selected_a)
    sitk_b = sitk.GetImageFromArray(selected_b)
    
    sitk.WriteImage(sitk_a, os.path.join(output_dir, name + '.a.nii'))
    sitk.WriteImage(sitk_b, os.path.join(output_dir, name + '.b.nii'))
    
    break
    
manifest = pd.DataFrame(manifest)
manifest.to_csv(os.path.join(output_dir, 'manifest.csv'), index=False)

63
0 /home/dwu/data/DECT/sinogram/sino_55_2.mat


In [21]:
for i, filename in enumerate(filenames):
    print (i, filename)

0 /home/dwu/data/DECT/sinogram/sino_3_2.mat
1 /home/dwu/data/DECT/sinogram/sino_13_2.mat
2 /home/dwu/data/DECT/sinogram/sino_46_2.mat
3 /home/dwu/data/DECT/sinogram/sino_14_2.mat
4 /home/dwu/data/DECT/sinogram/sino_47_2.mat
5 /home/dwu/data/DECT/sinogram/sino_16_2.mat
6 /home/dwu/data/DECT/sinogram/sino_49_2.mat
7 /home/dwu/data/DECT/sinogram/sino_17_2.mat
8 /home/dwu/data/DECT/sinogram/sino_4_2.mat
9 /home/dwu/data/DECT/sinogram/sino_18_2.mat
10 /home/dwu/data/DECT/sinogram/sino_53_2.mat
11 /home/dwu/data/DECT/sinogram/sino_54_2.mat
12 /home/dwu/data/DECT/sinogram/sino_19_2.mat
13 /home/dwu/data/DECT/sinogram/sino_55_2.mat
14 /home/dwu/data/DECT/sinogram/sino_20_2.mat
15 /home/dwu/data/DECT/sinogram/sino_56_2.mat
16 /home/dwu/data/DECT/sinogram/sino_21_2.mat
17 /home/dwu/data/DECT/sinogram/sino_57_2.mat
18 /home/dwu/data/DECT/sinogram/sino_22_2.mat
19 /home/dwu/data/DECT/sinogram/sino_58_2.mat
20 /home/dwu/data/DECT/sinogram/sino_23_2.mat
21 /home/dwu/data/DECT/sinogram/sino_59_2.mat
